In [106]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as p
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import scatterplot




database_name = 'telecom'
table_name= 'xdr_data'

connection_params = { "host": "localhost", "user": "postgres", "password": "admin",
                    "port": "5432", "database": database_name}

engine = create_engine(f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}")


## Loading the data and screening it

In [ ]:
# str or SQLAlchemy Selectable (select or text object)
sql_query = 'SELECT * FROM xdr_data'

df = pd.read_sql(sql_query, con= engine)

In [ ]:
# To get a concise summary of the dataset i.e. data types, missing data,..
df.info()
pd.set_option('display.max_rows', None)

In [ ]:
# Display the first 10 rows of the dataframe
df.head(10)

In [ ]:
#List of column names
print(df.columns)

In [ ]:
# Find out the missing values
df.isna().sum()

In [ ]:
# Generate descriptive statistics
df.describe()

In [ ]:
#Returning a tuple with dimensions of the dataframe
print(df.shape)

## Task 2 sub tasks.

In [ ]:
# Identifying the top 10 of handsets used by the customers.
top_10_of_handsets =df['Handset Type'].value_counts().head(10)
print("Top 10 Handsets:")
print(top_10_of_handsets)

In [ ]:
#Identify the top 3 handset manufacturers by count
top_3_manufacturers = df['Handset Manufacturer'].value_counts().head(3)

print("Top 3 Handset Manufacturers:")
print(top_3_manufacturers)

In [ ]:
#Find out the top 5 handsets in each of the top three manufactures
top_3_manufacturers = df['Handset Manufacturer'].value_counts().head(3).index.tolist()
filtered_df = df[df['Handset Manufacturer'].isin(top_3_manufacturers)]
top_5_handsets_per_manufacturer = filtered_df.groupby('Handset Manufacturer')['Handset Type'].value_counts().groupby('Handset Manufacturer').head(5)

print("Top 5 Handsets per Top 3 of the Handset Manufacturers:")
print(top_5_handsets_per_manufacturer)

In [ ]:
#Find out the bottom 5 handsets in each of the bottom three manufactures
bottom_3_manufacturers = df['Handset Manufacturer'].value_counts().tail(3).index.tolist()
filtered_df = df[df['Handset Manufacturer'].isin(bottom_3_manufacturers)]
bottom_5_handsets_per_manufacturer = filtered_df.groupby('Handset Manufacturer')['Handset Type'].value_counts().groupby('Handset Manufacturer').tail(5)

print("Botttom 5 Handsets per bottom 3 of the Handset Manufacturers:")
print(bottom_5_handsets_per_manufacturer)

In [ ]:
# Interpretation:
# The most used handset is Huawei B528S-23A.  It is also the only handset from the Huawei Manufacturer that appears in the top 10 most used handsets.
# I phone handsets seem to have the highest number of users as they take up 7 positions in top 10 mosted used handsets. 
# The top three handset manufacturers are Apple, Samsung, huawei respectively with each producing over 34400 handsets.
# In each the top 3 manufacturers, the leading handset also appears in the top 10 of the most used handsets. Also, all the top 5 handsets in the manufacturer, Apple, also 
# appear in the top 10 most used handsets overall.

# Recommendations:
# The marketing team should consider partnering with the handset manufactures, in a mutually benefitting way, in order to give promotions to customers using the top 10 handsets to enhance customer retention.
# They could also do the same for the least used handsets in order to broaden their customer base. 


# Task 2.1
## Overview of the users behaviour.

# number of xDR sessions per user

In [102]:
# Select columns pertaining to applications and user data
user_app_columns = ['Bearer Id', 'Handset Manufacturer', 'Handset Type', 'Social Media DL (Bytes)',
                    'Social Media UL (Bytes)', 'Google DL (Bytes)', 'Google UL (Bytes)',
                    'Email DL (Bytes)', 'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
                    'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
                    'Other DL (Bytes)', 'Other UL (Bytes)']

In [ ]:
df_user_apps = df[user_app_columns]

In [ ]:
# To count the number of xDR sessions per user

df_user_apps['Number of xDR Sessions'] = df_user_apps.groupby('Bearer Id')['Bearer Id'].transform('count')

In [ ]:
# Extracting the relevant columns for the result
result_columns = ['Bearer Id', 'Number of xDR Sessions']

result = df_user_apps[result_columns].drop_duplicates().reset_index(drop=True)
print(result)

In [ ]:
# Aggregate session duration per user
session_dur_per_user = df.groupby('Bearer Id')['Dur. (ms)'].sum().reset_index(name='Session Duration (ms)')

print(session_dur_per_user)

In [ ]:
# Aggregate the total download and upload data per user
total_data_per_user =pd.DataFrame (df.groupby('Bearer Id')[['Total DL (Bytes)', 'Total UL (Bytes)']].sum().reset_index())

print(total_data_per_user)

In [ ]:
# Define the columns related to applications
application_columns = ['Bearer Id', 'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
                        'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
                        'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
                         'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
                         'Gaming UL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)'
                        
                       ]

# Aggregate the total data volume per user and application
total_data_per_user_app = df[application_columns].groupby('Bearer Id').sum().reset_index()

print(total_data_per_user_app)

# Task 2.2

In [114]:
# Creating separate dataframes for columns with numbers and objects.
num_col = df.select_dtypes (include = ['number']).columns
cat_col = df.select_dtypes (include = ['object']).columns

df_num = df[num_col]
df_cat = df[cat_col]

print("Numerical Columns:", num_col)
print("Categorical Columns:", cat_col)

Numerical Columns: Index(['Bearer Id', 'Start ms', 'End ms', 'Dur. (ms)', 'IMSI', 'MSISDN/Number',
       'IMEI', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)',
       'Avg Bearer TP UL (kbps)', 'TCP DL Retrans. Vol (Bytes)',
       'TCP UL Retrans. Vol (Bytes)', 'DL TP < 50 Kbps (%)',
       '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)',
       'DL TP > 1 Mbps (%)', 'UL TP < 10 Kbps (%)',
       '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)',
       'UL TP > 300 Kbps (%)', 'HTTP DL (Bytes)', 'HTTP UL (Bytes)',
       'Activity Duration DL (ms)', 'Activity Duration UL (ms)', 'Dur. (ms).1',
       'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',
       'Nb of sec with Vol DL < 6250B', 'Nb of sec with Vol UL < 1

In [ ]:
# filling in missing values with mean
df_num = df_num.fillna(df_num.mean())

In [ ]:
# Checking for missing values
df_num.isna().sum()

In [ ]:
# Replacing outliers with mean

def replace_outliers_with_mean(df_num, threshold=3):
    for col in df_num.columns:
        z_scores = (df_num[col] - df_num[col].mean()) / df_num[col].std()
        outlier_mask = (z_scores > threshold) | (z_scores < -threshold)
        df_num[col][outlier_mask] = df_num[col].mean()
    return df_num

df_num = replace_outliers_with_mean(df_num)

In [ ]:
# Basic metrics
df_num.describe()

In [ ]:
correlation_matrix = df_num.corr()
print(correlation_matrix)

In [ ]:


columns_to_plot = ['Dur. (ms)', 'Avg RTT DL (ms)',
                   'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',  
                   'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
                    'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
                    'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
                    'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
                    'Gaming UL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)']

# Set up subplots
num_plots = len(columns_to_plot)
fig, axes = plt.subplots(nrows=num_plots, ncols=1, figsize=(10, 4 * num_plots))

# Plot histograms for the columns
for i, column in enumerate(columns_to_plot):
    axes[i].hist(df_num[column], bins=20, color='skyblue', edgecolor='black')
    axes[i].set_title(f'Histogram of {column}')
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('Frequency')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:

columns_to_plot = ['Dur. (ms)', 'Avg RTT DL (ms)',
                   'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
                   'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
                   'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
                   'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
                   'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
                   'Gaming UL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)']

# Set up subplots
num_plots = len(columns_to_plot)
fig, axes = plt.subplots(nrows=num_plots, ncols=1, figsize=(10, 4 * num_plots))

# Plot box plots for the columns
for i, column in enumerate(columns_to_plot):
    axes[i].boxplot(df_num[column])
    axes[i].set_title(f'Box Plot of {column}')
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('Value')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
df_num['Total Bytes Sum'] = df_num['Total DL (Bytes)'] + df_num['Total UL (Bytes)']
print(df_num['Total Bytes Sum'])

In [ ]:
#Define the social media columns and find their correlations to the Total Bytes

application_columns2 = ['Social Media DL (Bytes)', 'Social Media UL (Bytes)',
                        'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
                        'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
                         'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
                         'Gaming UL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)'
                        
                       ]


correlations = df_num[application_columns2 + ['Total Bytes Sum']]. corr()
correlations

In [ ]:

for app_column in application_columns2:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='Total Bytes Sum', y=app_column, data=df_num)  
    plt.title(f'Scatter Plot: {app_column} vs Total Bytes Sum')
    plt.xlabel('Total Bytes Sum')
    plt.ylabel(app_column)
    plt.show()


In [ ]:
correlation_matrix = df_num[application_columns2 + ['Total Bytes Sum']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.show()      
            

In [ ]:
df_num['Total Duration'] = df_num['Dur. (ms)']

# Creating decile classes based on total duration.
df_num['Duration Decile'] = pd.qcut(df_num['Total Duration'], q=[0, 0.2, 0.4, 0.6, 0.8, 1], labels=False, precision=0)

# Group by 'MSISDN/Number' and 'Duration Decile', then compute the sum of 'Total Bytes Sum' for each group
decile_totals = df_num.groupby(['MSISDN/Number', 'Duration Decile'])['Total Bytes Sum'].sum().reset_index()

# Get the top five decile classes
top_five_deciles = decile_totals.groupby('Duration Decile')['Total Bytes Sum'].sum().nlargest(5).index

# Filter the DataFrame for the top five decile classes
top_five_deciles_data = decile_totals[decile_totals['Duration Decile'].isin(top_five_deciles)]

# Display the result
print(top_five_deciles_data)

In [ ]:
# Subset the DataFrame with the application columns.
subset_df = df_num[application_columns2]

# Calculate the correlation matrix
correlation_matrix = subset_df.corr

print(correlation_matrix)